## IMPORT

In [57]:
import twitter
import spacy
import nltk
from spacy import displacy
import markovify
import gzip
import re
import json
import spacy

In [51]:
nlp = spacy.load('en_core_web_lg')

## RETRIEVE TWEETS

In [3]:
twitter_credentials = {
    'consumer_key':        'wbz78wFd0ywcShiTvqgDUV2ry',
    'consumer_secret':     '2qj0P3fygqa0n2LqU6M8LV485OWIAvXWEQOEVLWFNUBdKDcgjz',
    'access_token_key':    '80578720-t7bH4zwD6Q6sUQEFeCb8211wH04Y9ul0EWECo2ofU',
    'access_token_secret': 'PSsX8R4agpxAII9XCYHqE74KObPRWfl9tdG4Xd07olOn6'
}

In [4]:
twapi = twitter.Api(consumer_key=twitter_credentials['consumer_key'],
                    consumer_secret=twitter_credentials['consumer_secret'],
                    access_token_key=twitter_credentials['access_token_key'],
                    access_token_secret=twitter_credentials['access_token_secret'],
                    tweet_mode='extended')

In [5]:
#  Retrieve as much tweet history as possible for training purposes
timeline = []
max_id = None
while True:
    print('max_id for iteration: {}'.format(max_id))
    tweets = twapi.GetUserTimeline(screen_name='realdonaldtrump', include_rts=False, count=200, max_id=max_id)
    timeline += tweets
    
    next_max_id = min(tweets, key=lambda t: t.id).id
    if next_max_id == max_id:
        break
    max_id = next_max_id

max_id for iteration: None
max_id for iteration: 1167771830947590144
max_id for iteration: 1164852076930879493
max_id for iteration: 1162002141172633600
max_id for iteration: 1157757443901972480
max_id for iteration: 1154112475002036234
max_id for iteration: 1150400995177959427
max_id for iteration: 1148016422783803392
max_id for iteration: 1143601087238172672
max_id for iteration: 1138967829418565632
max_id for iteration: 1134276359260389377
max_id for iteration: 1130205328522522627
max_id for iteration: 1127210559059972099
max_id for iteration: 1124653429366906882
max_id for iteration: 1121044792945926144
max_id for iteration: 1116502929542922241
max_id for iteration: 1110200331328208897
max_id for iteration: 1107001687208878085


In [12]:
timeline.reverse()
timeline_abridged = [{k:v for k,v in elem.AsDict().items() if k in ['created_at', 'full_text', 'id']} 
                      for elem in timeline]

In [29]:
#  Save abridged 
with open('/Users/liangjh/workspace/corpii/tweets-trump.json', 'w') as f:
    json.dump({'data': timeline_abridged}, f)

In [42]:
timeline_text_sanitized = [re.sub(r'(http|https)?:\/\/.*[\r\n]*', '', tline['full_text']) 
                           for tline in timeline_abridged]

In [44]:
timeline_merged_corpus = ' '.join(timeline_text_sanitized)

In [9]:
# with open('/Users/liangjh/workspace/corpii/tweets-trump.txt') as f:
#     tweets_raw_curr = f.read()

In [ ]:
#  Parsed document

In [48]:
doc = nlp(timeline_merged_corpus)

In [49]:
len(doc)

80415

In [73]:
# tweets_combined = ' '.join(tweets_raw)
# doc = nlp(tweets_combined)
# len(tweets_raw)
# len([token for token in doc])

In [52]:
class POSifiedText(markovify.Text):
    def word_split(self, sentence):
        return ['::'.join((word.orth_, word.pos_)) for word in nlp(sentence)]
    def word_join(self, words):
        sentence = ' '.join(word.split('::')[0] for word in words)
        return sentence

In [53]:
tweet_model = POSifiedText(timeline_merged_corpus, state_size=2)

In [60]:
# for i in range(20):
#     print(tweet_model.make_short_sentence(max_chars=150))

In [59]:
with gzip.GzipFile('/Users/liangjh/workspace/corpii/trumpspire-model-02.gz', 'w') as fo:
    fo.write(json.dumps(tweet_model.to_json()).encode('utf-8'))